<h1 style="text-align: center;"> Model Tuning </p>

<a id='Table-of-Contents'></a>


##  Table of Contents
1. [Required Libraries](#Required-Libraries)
2. [Load Data](#Load-Data)
3. 

## Required Libraries

[[ go back to the top ]](#Table-of-contents)

This notebook uses several Python libraries such as:

In [63]:
# Load required packages 
import datetime
from datetime import timedelta
import numpy as np
import pandas as pd

# Visuals
import matplotlib.pyplot as plt
import seaborn as sns

# Time-Series
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARMA, ARIMA

from sklearn.model_selection import train_test_split  

from scipy import signal
import scipy.stats as stats

from Functions import LOAD_DATA

import warnings
warnings.filterwarnings("ignore")

<a id='Load-Data'></a>

---
## Load Data

[[ go back to the top ]](#Table-of-Contents)

In [64]:
# Load Data Function
def LOAD_DATA(filepath, filename):
    # Read CSV files
    if filename.endswith('.csv'):
        new_df = pd.read_csv(filepath+filename)

    # Read Excel files
    elif filename.endswith('.xlsx'):
        new_df = pd.read_excel(filepath+filename)
    print(type(new_df.index))
    if type(new_df.index) != pd.core.indexes.datetimes.DatetimeIndex:
        for col in new_df.columns:
            if col.lower().find('date') != -1:
                print(f"TIMESTAMP FOUND! '{col}'")
                print()
                new_df['date'] = pd.to_datetime(new_df[col]) # format = '%Y/%m/%d'
                new_df.set_index('date', inplace = True)
                # If datetime col was already == 'date', no need to drop col after set_index, otherwise...
                if col != 'date':
                    new_df.drop(columns = col, inplace = True)
                
    # Try to identify the date column
    elif type(new_df.index) == pd.core.indexes.datetimes.DatetimeIndex:
        print('Index already in datetime')
        
    display(new_df.info())
    return new_df

In [65]:
filepath = '../../'
filename = 'lagged_wmobility_df.csv'
df = LOAD_DATA(filepath, filename)

<class 'pandas.core.indexes.range.RangeIndex'>
TIMESTAMP FOUND! 'date'

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 243 entries, 2020-02-19 to 2021-02-25
Data columns (total 85 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   phillips_66                            243 non-null    float64
 1   bp_plc                                 243 non-null    float64
 2   valero_energy_corporation              243 non-null    float64
 3   chevron_corporation                    243 non-null    float64
 4   occidental_petroleum_corporation       243 non-null    float64
 5   marathon_oil_corporation               243 non-null    float64
 6   pioneer_natural_resources_company      243 non-null    float64
 7   conocophillips                         243 non-null    float64
 8   exxon_mobil_corporation                243 non-null    float64
 9   marathon_petroleum_corporation         243 non-null

None

In [66]:
df.isna().sum().sum()

0

In [67]:
df

,phillips_66,bp_plc,valero_energy_corporation,chevron_corporation,occidental_petroleum_corporation,marathon_oil_corporation,pioneer_natural_resources_company,conocophillips,exxon_mobil_corporation,marathon_petroleum_corporation,...,lag_i_g_carbon_footprint,lag_i_g_emissions,lag_i_g_epa,lag_i_g_greenhouse,lag_i_g_hurricane_storm,lag_i_g_pollution,lag_i_g_sanction,lag_i_g_solar,lag_i_g_turbine,lag_i_g_vacation
date,,,,,,,,,,,,,,,,,,,,,
2020-02-19,88.36,36.25,83.23,110.74,42.51,10.47,141.17,59.26,60.34,57.69,...,-0.1,-0.1,-0.550000,-0.200000,0.000000,-0.500000,-0.427273,-0.175000,0.000000,-0.114286
2020-02-20,90.19,35.98,84.17,109.81,42.97,10.25,142.53,58.88,59.86,60.26,...,0.0,0.0,0.000000,-0.366667,0.000000,-0.225000,-0.120000,-0.350000,0.000000,-0.200000
2020-02-21,89.25,35.36,82.90,109.01,42.12,10.11,142.25,58.44,59.13,59.13,...,0.3,0.3,0.500000,-0.350000,0.000000,-0.200000,-0.366667,0.033333,0.000000,-0.600000
2020-02-24,86.56,34.06,78.10,104.71,39.48,9.46,135.97,56.38,56.36,55.87,...,0.0,0.0,0.000000,0.000000,0.000000,-0.275000,-0.220000,0.000000,0.000000,0.200000
2020-02-25,82.81,32.93,74.44,100.71,36.19,8.92,130.09,53.83,54.20,53.12,...,0.0,0.0,0.300000,-0.050000,0.000000,-0.466667,-0.433333,0.000000,0.000000,-0.133333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-19,82.31,22.87,71.53,95.80,25.42,9.45,133.94,48.42,52.37,53.00,...,0.2,0.2,-0.180000,-0.066667,0.000000,-0.287500,-0.428571,-0.040000,-0.568421,-0.100000
2021-02-22,83.96,23.63,74.25,98.39,26.47,10.20,139.47,50.88,54.30,54.85,...,-0.2,-0.2,-0.400000,-0.283333,0.000000,-0.290909,-0.424138,-0.157143,-0.350000,-0.288000
2021-02-23,85.53,24.23,75.81,99.63,26.06,11.16,145.24,52.10,55.05,55.77,...,0.0,0.0,-0.200000,0.500000,-0.245455,-0.266667,-0.490476,0.071429,-0.300000,-0.117647


### Fit an ARIMA Model

In [68]:
df.columns

Index(['phillips_66', 'bp_plc', 'valero_energy_corporation',
       'chevron_corporation', 'occidental_petroleum_corporation',
       'marathon_oil_corporation', 'pioneer_natural_resources_company',
       'conocophillips', 'exxon_mobil_corporation',
       'marathon_petroleum_corporation', 'lag_phillips_66', 'lag_bp_plc',
       'lag_valero_energy_corporation', 'lag_chevron_corporation',
       'lag_occidental_petroleum_corporation', 'lag_marathon_oil_corporation',
       'lag_pioneer_natural_resources_company', 'lag_conocophillips',
       'lag_exxon_mobil_corporation', 'lag_marathon_petroleum_corporation',
       'lag_dow_jones_transportation_average',
       'lag_dow_jones_composite_average', 'lag_s&p_500',
       'lag_dow_jones_industrial_average', 'lag_dow_jones_utility_average',
       'lag_workplaces', 'lag_retail_and_recreation',
       'lag_grocery_and_pharmacy', 'lag_residential', 'lag_transit_stations',
       'lag_parks', 'lag_Value_ger_walk', 'lag_Value_ger_drive',
      

In [69]:
# Set-Up Target-Variables: NYSE Closing Price
targets = ['phillips_66',
           'bp_plc',
           'valero_energy_corporation',
           'chevron_corporation',
           'occidental_petroleum_corporation',
           'marathon_oil_corporation',
           'pioneer_natural_resources_company',
           'conocophillips',
           'exxon_mobil_corporation',
           'marathon_petroleum_corporation']

# DataFrame of 10 large Oil Companies
target_df = df[targets]

In [70]:
# Create Train/Test-Split
# Typical: test_size = most recent 20%(small datasets) - 33%(large datasets)
X_train, X_valid, y_train, y_valid = train_test_split(df.drop(columns = targets),  
                                                    df['phillips_66'],  
                                                    test_size = 0.2, 
                                                    shuffle = False) 

In [71]:
# Check shape and verify proper split above 
print(X_train.shape) 
print(X_valid.shape)  
print(y_train.shape) 
print(y_valid.shape)

(194, 75)
(49, 75)
(194,)
(49,)


In [72]:
# Before fitting a model in statsmodels (Hint: Add the intercept)  
X_train = sm.add_constant(X_train) 
X_valid = sm.add_constant(X_valid)
# Check that all is copacetic  
X_train.head()

,const,lag_phillips_66,lag_bp_plc,lag_valero_energy_corporation,lag_chevron_corporation,lag_occidental_petroleum_corporation,lag_marathon_oil_corporation,lag_pioneer_natural_resources_company,lag_conocophillips,lag_exxon_mobil_corporation,...,lag_i_g_carbon_footprint,lag_i_g_emissions,lag_i_g_epa,lag_i_g_greenhouse,lag_i_g_hurricane_storm,lag_i_g_pollution,lag_i_g_sanction,lag_i_g_solar,lag_i_g_turbine,lag_i_g_vacation
date,,,,,,,,,,,,,,,,,,,,,
2020-02-19,1.0,88.00,36.09,82.52,110.24,41.56,10.30,135.56,58.30,59.88,...,-0.1,-0.1,-0.55,-0.200000,0.0,-0.500000,-0.427273,-0.175000,0.0,-0.114286
2020-02-20,1.0,88.36,36.25,83.23,110.74,42.51,10.47,141.17,59.26,60.34,...,0.0,0.0,0.00,-0.366667,0.0,-0.225000,-0.120000,-0.350000,0.0,-0.200000
2020-02-21,1.0,90.19,35.98,84.17,109.81,42.97,10.25,142.53,58.88,59.86,...,0.3,0.3,0.50,-0.350000,0.0,-0.200000,-0.366667,0.033333,0.0,-0.600000
2020-02-24,1.0,89.25,35.36,82.90,109.01,42.12,10.11,142.25,58.44,59.13,...,0.0,0.0,0.00,0.000000,0.0,-0.275000,-0.220000,0.000000,0.0,0.200000
2020-02-25,1.0,86.56,34.06,78.10,104.71,39.48,9.46,135.97,56.38,56.36,...,0.0,0.0,0.30,-0.050000,0.0,-0.466667,-0.433333,0.000000,0.0,-0.133333


In [73]:
# statsmodels can't handle missing values
X_train.isnull().sum().sum(), X_valid.isnull().sum().sum(), y_train.isnull().sum().sum(), y_valid.isnull().sum().sum()

(0, 0, 0, 0)

In [74]:
# Starting AIC, p, and q.
best_aic = 99 * (10 ** 16) # Just a really big number
best_p = 0
best_q = 0

# Use nested for loop to iterate over values of p and q.
for p in range(5):
    for q in range(5):
        # Insert try and except statements.
        try:
            # Fitting an ARIMA(p, 1, q) model.
            print(f'Attempting to fit ARIMA({p},1,{q})') # Default d=1 is given
            
            # Instantiate ARIMA model
            arima = ARIMA(endog = y_train.astype(float).dropna(), # endog = Y variable ,"enogenous param"
                          order = (p,1,q)) # values of p, d, q
            
            # Fit ARIMA model.
            model = arima.fit()

            # Print out AIC for ARIMA(p, 1, q) model.
            print(f'The AIC for ARIMA({p},1,{q}) is: {model.aic}')

            # Is my current model's AIC better than our best_aic?
            if model.aic < best_aic: # Update best params if new and improved score
                
                # If so, let's overwrite best_aic, best_p, and best_q.
                best_aic = model.aic
                best_p = p
                best_q = q

        except:
            pass
print()
print()
print('MODEL FINISHED!')
print(f'Our model that minimizes AIC on the training data is the ARIMA({best_p},1,{best_q}).')
print(f'This model has an AIC of {best_aic}.')

Attempting to fit ARIMA(0,1,0)
The AIC for ARIMA(0,1,0) is: 951.6998547025115
Attempting to fit ARIMA(0,1,1)
The AIC for ARIMA(0,1,1) is: 953.3031635328748
Attempting to fit ARIMA(0,1,2)
The AIC for ARIMA(0,1,2) is: 955.0060752405045
Attempting to fit ARIMA(0,1,3)
The AIC for ARIMA(0,1,3) is: 956.9022421570714
Attempting to fit ARIMA(0,1,4)
The AIC for ARIMA(0,1,4) is: 958.5193520310094
Attempting to fit ARIMA(1,1,0)
The AIC for ARIMA(1,1,0) is: 953.2722922696566
Attempting to fit ARIMA(1,1,1)
The AIC for ARIMA(1,1,1) is: 954.7027571922815
Attempting to fit ARIMA(1,1,2)
The AIC for ARIMA(1,1,2) is: 956.0083614155967
Attempting to fit ARIMA(1,1,3)
The AIC for ARIMA(1,1,3) is: 957.9833749226788
Attempting to fit ARIMA(1,1,4)
The AIC for ARIMA(1,1,4) is: 959.7859321993653
Attempting to fit ARIMA(2,1,0)
The AIC for ARIMA(2,1,0) is: 955.0106725087346
Attempting to fit ARIMA(2,1,1)
The AIC for ARIMA(2,1,1) is: 956.0492599776812
Attempting to fit ARIMA(2,1,2)
The AIC for ARIMA(2,1,2) is: 954.

In [75]:
arima.params

AttributeError: 'ARIMA' object has no attribute 'params'

In [78]:
type(y_valid.index[0])

pandas._libs.tslibs.timestamps.Timestamp

In [90]:
y_valid['2020-12-15']

68.25

In [92]:
type(y_valid)

pandas.core.series.Series

In [82]:
y_valid.index[-1]

Timestamp('2021-02-25 00:00:00')

In [ ]:
str(pd.to_datetime('2021-02-25'))

In [ ]:
y_train.astype(float)

In [ ]:
len(y_valid)

In [81]:
y_valid.index

DatetimeIndex(['2020-12-15', '2020-12-16', '2020-12-17', '2020-12-18',
               '2020-12-21', '2020-12-22', '2020-12-23', '2020-12-24',
               '2020-12-28', '2020-12-29', '2020-12-30', '2020-12-31',
               '2021-01-04', '2021-01-05', '2021-01-06', '2021-01-07',
               '2021-01-08', '2021-01-11', '2021-01-12', '2021-01-13',
               '2021-01-14', '2021-01-15', '2021-01-19', '2021-01-20',
               '2021-01-21', '2021-01-22', '2021-01-25', '2021-01-26',
               '2021-01-27', '2021-01-28', '2021-01-29', '2021-02-01',
               '2021-02-02', '2021-02-03', '2021-02-04', '2021-02-05',
               '2021-02-08', '2021-02-09', '2021-02-10', '2021-02-11',
               '2021-02-12', '2021-02-16', '2021-02-17', '2021-02-18',
               '2021-02-19', '2021-02-22', '2021-02-23', '2021-02-24',
               '2021-02-25'],
              dtype='datetime64[ns]', name='date', freq=None)

In [100]:
# MODEL FINISHED!
# Our model that minimizes AIC on the training data is the ARIMA(0,1,0).
# This model has an AIC of 951.67

# Instantiate best model found by GridSearch above
model = ARIMA(#endog = np.array(y_train),  # endog = endogenous "Target" Variable
              endog = y_train.astype(float).dropna(),
              dates = y_train.index, # pass in training index using the ARIMA dates parameter
              freq = 'B',
              order = (0,1,0))

# Fit ARIMA model.
arima = model.fit()
# Generate predictions based on valid set.
preds = model.predict(params = arima.params,
                      start = y_valid.index[0],
                      end = y_valid[-1])

# Plot data.
plt.figure(figsize=(10,6))

# Plot training data.
#plt.plot(y_train.index, pd.DataFrame(y_train), color = 'blue')
plt.plot(y_train.index, pd.DataFrame(y_train).diff(), color = 'blue')
# Plot validing data.
#plt.plot(y_valid.index, pd.DataFrame(y_valid), color = 'orange')
plt.plot(y_valid.index, pd.DataFrame(y_valid).diff(), color = 'orange')
# Plot predicted valid values.
plt.plot(y_valid.index, preds, color = 'green')

plt.title(label = 'Once-Differenced Global Mean Temperature with ARIMA(3, 1, 3) Predictions', fontsize=16)

ValueError: The given frequency argument could not be matched to the given index.

NOTES
https://stackoverflow.com/questions/56007991/predict-time-series-with-statsmodels-var-and-encountering-valueerror
LSSN 10.02 & 10.03 GA

In [ ]:
fig, ax = plt.subplots(nrows = 3,
                       ncols = 1,
                       figsize=(20,10), 
                       sharex = True)

df_sub = df[['workplaces']]

# Element-wise subtraction w/ a Series, for set_title("RSS")
df_diff = df_differenced.workplaces

fsize = 15
p = 5; d = 1; q = 1

for nax in range(3):
    if nax == 0:
        # Instantiate the AR Model, choose "p" (p, d = 1, q = 0)
        model = ARIMA(df_sub, order=(p,d,0))
        # Fit the AR Model
        preds_AR = model.fit(disp=-1)
        ax[nax].plot(preds_AR.fittedvalues, label = f'AR({p}) Predictions')
        # Residual Sum of Squares
        ax[nax].set_title(f'AR({p}) Model - RSS: %.4f'%sum((preds_AR.fittedvalues - df_diff)**2),
                          fontsize = fsize)
        
    elif nax == 1:
        # MA Model, choose "q" (p = 0, d = 1, q)
        model = ARIMA(df_sub, order=(0,d,q))
        preds_MA = model.fit(disp=-1)
        ax[nax].plot(preds_MA.fittedvalues, label = f'MA({q}) Predictions')
        # Residual Sum of Squares
        ax[nax].set_title(f'MA({q}) Model - RSS: %.4f'%sum((preds_MA.fittedvalues - df_diff)**2),
                          fontsize = fsize)
        
    elif nax == 2:
        # ARIMA model, choose "p" & "q" (p, d, q)
        model = ARIMA(df_sub, order=(p,d,q))
        preds_ARIMA = model.fit(disp=-1)
        ax[nax].plot(preds_ARIMA.fittedvalues, label = f'ARIMA({p},{d},{q}) Predictions')
        # Residual Sum of Squares
        ax[nax].set_title(f'ARIMA({p},{d},{q}) Model - RSS: %.4f'%sum((preds_ARIMA.fittedvalues - df_diff)**2),
                          fontsize = fsize)
        
    ax[nax].plot(df_diff, label = 'Original')    
    ax[nax].legend()
    #ax[nax].set_xlabel('Timestamp')
    ax[nax].set_ylabel('Mobility (% Change)', fontsize = fsize)
    
plt.tight_layout()